In [2]:
pip install nltk

In [6]:
import nltk

In [8]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
import pandas as pd

In [12]:
dt=pd.read_csv("spam.csv",encoding='windows-1252')

In [14]:
import chardet
with open("spam.csv",'rb') as rawdata:
  result=chardet.detect(rawdata.read(100000))
result

{'confidence': 0.73, 'encoding': 'Windows-1252', 'language': ''}

In [16]:
dt.head() #gives top 5 rows

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
dt['spam']=dt['type'].map({'spam':1,'ham':0}).astype(int)

In [20]:
dt.head()

,type,text,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [21]:
t=len(dt['type'])
print("NO OF ROWS IN REVIEW COLUMN:",t)

NO OF ROWS IN REVIEW COLUMN: 116


In [22]:
#Tokenization
dt['text'][2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [24]:
def tokenizer(text):
  return text.split()

In [25]:
dt['text']=dt['text'].apply(tokenizer)

In [26]:
dt['text'][2]

['Free',
 'entry',
 'in',
 '2',
 'a',
 'wkly',
 'comp',
 'to',
 'win',
 'FA',
 'Cup',
 'final',
 'tkts',
 '21st',
 'May',
 '2005.',
 'Text',
 'FA',
 'to',
 '87121',
 'to',
 'receive',
 'entry',
 'question(std',
 'txt',
 "rate)T&C's",
 'apply',
 "08452810075over18's"]

In [28]:
#Stemming
from nltk.stem import SnowballStemmer
porter=SnowballStemmer("english",ignore_stopwords=False)

In [29]:
def stem_it(text):
  return [porter.stem(word) for word in text]

In [30]:
dt['text']=dt['text'].apply(stem_it)

In [31]:
dt['text'][2]

['free',
 'entri',
 'in',
 '2',
 'a',
 'wkli',
 'comp',
 'to',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 '21st',
 'may',
 '2005.',
 'text',
 'fa',
 'to',
 '87121',
 'to',
 'receiv',
 'entri',
 'question(std',
 'txt',
 'rate)t&c',
 'appli',
 '08452810075over18']

In [33]:
#lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [36]:
def lemmit_it(text):
  return [lemmatizer.lemmatize(word,pos='a') for word in text]


In [37]:
dt['text']=dt['text'].apply(lemmit_it)

In [38]:
dt['text'][2]

['free',
 'entri',
 'in',
 '2',
 'a',
 'wkli',
 'comp',
 'to',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 '21st',
 'may',
 '2005.',
 'text',
 'fa',
 'to',
 '87121',
 'to',
 'receiv',
 'entri',
 'question(std',
 'txt',
 'rate)t&c',
 'appli',
 '08452810075over18']

In [42]:
#stopwords
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

In [41]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
def stop_it(text):
  review=[word for word in text if not word in stop_words]
  return review

In [44]:
dt['text']=dt['text'].apply(stop_it)

In [45]:
dt['text'][2]

['free',
 'entri',
 '2',
 'wkli',
 'comp',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 '21st',
 'may',
 '2005.',
 'text',
 'fa',
 '87121',
 'receiv',
 'entri',
 'question(std',
 'txt',
 'rate)t&c',
 'appli',
 '08452810075over18']

In [46]:
dt.head()

,type,text,spam
0,ham,"[go, jurong, point,, crazy.., avail, onli, bug...",0
1,ham,"[ok, lar..., joke, wif, u, oni...]",0
2,spam,"[free, entri, 2, wkli, comp, win, fa, cup, fin...",1
3,ham,"[u, dun, say, earli, hor..., u, c, alreadi, sa...",0
4,ham,"[nah, think, goe, usf,, live, around, though]",0


In [47]:
dt['text']=dt['text'].apply(' '.join)

In [48]:
dt.head()

,type,text,spam
0,ham,"go jurong point, crazy.. avail onli bugi n gre...",0
1,ham,ok lar... joke wif u oni...,0
2,spam,free entri 2 wkli comp win fa cup final tkts 2...,1
3,ham,u dun say earli hor... u c alreadi say...,0
4,ham,"nah think goe usf, live around though",0


In [50]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
y=dt.spam.values
x=tfidf.fit_transform(dt['text'])


In [51]:
x

<116x712 sparse matrix of type '<class 'numpy.float64'>'
	with 1081 stored elements in Compressed Sparse Row format>

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1,shuffle=False)

In [55]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [56]:
from sklearn.metrics import accuracy_score
acc_log=accuracy_score(y_pred,y_test)*100
print("Accuracy:",acc_log)

Accuracy: 91.42857142857143
